### Basic library imports

In [2]:
import os
import pandas as pd

### Read Dataset

In [11]:
DATASET_FOLDER = '../dataset/'
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))

### Run Sanity check using src/sanity.py

In [12]:
!python sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out.csv

Parsing successfull for file: ../dataset/sample_test_out.csv


In [13]:
!python sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out_fail.csv

Error: Invalid unit [lbs] found in 6.75 lbs. Allowed units: {'cubic foot', 'yard', 'foot', 'decilitre', 'gram', 'volt', 'ounce', 'milligram', 'gallon', 'millivolt', 'kilogram', 'pint', 'centilitre', 'microgram', 'fluid ounce', 'pound', 'metre', 'ton', 'millimetre', 'quart', 'imperial gallon', 'microlitre', 'millilitre', 'watt', 'cubic inch', 'centimetre', 'litre', 'inch', 'cup', 'kilowatt', 'kilovolt'}


### Download images

In [14]:
from utils import download_images
download_images(sample_test['image_link'], '../images')

100%|██████████| 88/88 [00:12<00:00,  7.20it/s]


In [9]:
assert len(os.listdir('../images')) > 0

In [10]:
rm -rf ../images

In [4]:
train_df = pd.read_csv('/Users/shubham/Downloads/7th Sem/Amazon ML/student_resource 3/dataset/train.csv')
test_df = pd.read_csv('/Users/shubham/Downloads/7th Sem/Amazon ML/student_resource 3/dataset/test.csv')

In [5]:
#Text Extraction with OCR
import easyocr

reader = easyocr.Reader(['en'])

def extract_text_from_image(image_path):
    result = reader.readtext(image_path)
    text = " ".join([res[1] for res in result])
    return text
sample_image_path = '../images/41nblnEkJ3L.jpg'
extracted_text = extract_text_from_image(sample_image_path)
print(extracted_text)

/Users/shubham/Downloads/7th Sem/Amazon ML/shubham/lib/python3.12/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(co

LED OSRAM 55w 40w 470 Im ar White B22d


In [6]:
#Feature Extraction with CNN
# main.py
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Load a pre-trained ResNet model
resnet = models.resnet50(pretrained=True)
resnet.eval()

# Image preprocessing function
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def extract_image_features(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        features = resnet(image)
    return features.numpy().flatten()

# Example usage
image_features = extract_image_features(sample_image_path)
print(image_features.shape)

/Users/shubham/Downloads/7th Sem/Amazon ML/shubham/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/shubham/Downloads/7th Sem/Amazon ML/shubham/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(1000,)


In [8]:
#Data Processing
import re

def extract_entity_value(extracted_text):
    # Simple regex pattern to extract "number unit" format
    pattern = r'(\d+(\.\d+)?)\s*(gram|kilogram|centimetre|inch|ounce)'
    match = re.search(pattern, extracted_text)
    if match:
        return f"{match.group(1)} {match.group(3)}"
    return ""

# Example usage
entity_value = extract_entity_value(extracted_text)
print(entity_value)

In [9]:
# Model Training
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Prepare training data
X_image_features = [extract_image_features(f'train_images/{idx}.jpg') for idx in train_df['index']]
X_text_features = [extract_text_from_image(f'train_images/{idx}.jpg') for idx in train_df['index']]

# Encode the target variable
y = train_df['entity_value']
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train_img, X_test_img, X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_image_features, X_text_features, y_encoded, test_size=0.2, random_state=42)

# Combine image and text features in a pipeline
model_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Train the model using image features only for simplicity
model_pipeline.fit(X_train_text, y_train)

# Evaluate the model
print(f"Training accuracy: {model_pipeline.score(X_test_text, y_test)}")

KeyError: 'index'